In [2]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import unidecode

In [3]:
df_resultados2 = pd.read_excel('controle_por_tratamento_e_por_CID.xlsx')

In [4]:
df_filtrado = df_resultados2[df_resultados2['Órgão de ATS'] != 'NatJus Nacional']

# Agrupar pelas colunas 'Tecnologia' e 'CID' e contar os valores únicos de 'Órgão de ATS'
tecnologias_validas = df_filtrado.groupby(['Tecnologia', 'CID'])['Órgão de ATS'].nunique()
tecnologias_validas = tecnologias_validas[tecnologias_validas >= 2].index

# Filtrar o DataFrame original considerando as duas colunas 'Tecnologia' e 'CID'
df_filtrado = df_filtrado.set_index(['Tecnologia', 'CID'])
df_filtrado = df_filtrado.loc[df_filtrado.index.isin(tecnologias_validas)].reset_index()

In [5]:
# Identificar o órgão mais e menos favorável para cada combinação de Tecnologia e CID
resultados = []
for (tecnologia, cid), grupo in df_filtrado.groupby(['Tecnologia', 'CID']):
    # Encontrar os valores máximos e mínimos
    max_fav = grupo[grupo['Favorável'] == grupo['Favorável'].max()]
    min_fav = grupo[grupo['Favorável'] == grupo['Favorável'].min()]
    
    # Tratar empates para o máximo
    if len(max_fav) > 1:
        orgao_max = "/".join(max_fav['Órgão de ATS'])
        num_max = "/".join(map(str, max_fav['Nº de pareceres']))
        valor_max = max_fav['Favorável'].iloc[0]
    else:
        orgao_max = max_fav['Órgão de ATS'].iloc[0]
        num_max = max_fav['Nº de pareceres'].iloc[0]
        valor_max = max_fav['Favorável'].iloc[0]
    
    # Tratar empates para o mínimo
    if len(min_fav) > 1:
        orgao_min = "/".join(min_fav['Órgão de ATS'])
        num_min = "/".join(map(str, min_fav['Nº de pareceres']))
        valor_min = min_fav['Favorável'].iloc[0]
    else:
        orgao_min = min_fav['Órgão de ATS'].iloc[0]
        num_min = min_fav['Nº de pareceres'].iloc[0]
        valor_min = min_fav['Favorável'].iloc[0]
    
    # Calcular o total de pareceres emitidos sobre a combinação de tecnologia e CID
    total_pareceres = grupo['Nº de pareceres'].sum()
    
    resultados.append({
        'Tecnologia': tecnologia,
        'CID': cid,
        'Total de pareceres': total_pareceres,
        'Órgão de ATS + Favorável': orgao_max,
        'Nº de pareceres do Órgão de ATS + favorável': num_max,
        'Valor em favorável do Órgão de ATS + favorável': valor_max,
        'Órgão de ATS - Favorável': orgao_min,
        'Nº de pareceres do Órgão de ATS - favorável': num_min,
        'Valor em favorável do Órgão de ATS - favorável': valor_min,
        'Diferença entre o órgão + favorável e o - favorável': valor_max - valor_min
    })

# Criar o novo DataFrame com os resultados
df_meds = pd.DataFrame(resultados)
df_meds.to_excel('difs.xlsx')

In [21]:
# Calcular a quantidade de linhas onde a diferença foi de 100
dif_100 = df_meds[df_meds['Diferença entre o órgão + favorável e o - favorável'] == 100].shape[0]

# Calcular a quantidade de linhas onde a diferença foi igual ou superior a 90%
dif_80 = df_meds[df_meds['Diferença entre o órgão + favorável e o - favorável'] >= 80].shape[0]

# Calcular a quantidade de linhas onde a diferença foi igual ou superior a 50%
dif_50 = df_meds[df_meds['Diferença entre o órgão + favorável e o - favorável'] >= 50].shape[0]

# Calcular a quantidade de linhas onde a diferença foi igual ou superior a 50%
dif_20 = df_meds[df_meds['Diferença entre o órgão + favorável e o - favorável'] >= 20].shape[0]

# Imprimir os resultados
print(f"Diferença de 100: {dif_100} linhas")
print(f"Diferença igual ou superior a 80%: {dif_80} linhas")
print(f"Diferença igual ou superior a 50%: {dif_50} linhas")
print(f"Diferença igual ou superior a 20%: {dif_20} linhas")
print(len(df_meds))

Diferença de 100: 28 linhas
Diferença igual ou superior a 80%: 67 linhas
Diferença igual ou superior a 50%: 119 linhas
Diferença igual ou superior a 20%: 168 linhas
247


In [22]:
# Remove valores nulos e divide cada entrada da coluna em uma lista de elementos únicos (sem repetição na mesma linha)
informacoes_divididas = df_meds.loc[df_meds['Diferença entre o órgão + favorável e o - favorável'] >= 80]['Órgão de ATS + Favorável'].apply(lambda x: list(set(x.split('/'))))

# Cria uma lista plana com todas as informações únicas de todas as linhas
todas_informacoes = [info for sublist in informacoes_divididas for info in sublist]

# Conta a frequência de cada informação única
contagem_informacoes = pd.Series(todas_informacoes).value_counts()

contagem_informacoes

NatJus BA     39
NatJus SE     10
NatJus MS      7
NatJus DFT     6
NatJus MA      5
NatJus RJ      3
NatJus PR      3
NatJus SP      2
NatJus GO      1
NatJus PE      1
TJRS           1
DMJ            1
Name: count, dtype: int64

In [23]:
# Remove valores nulos e divide cada entrada da coluna em uma lista de elementos únicos (sem repetição na mesma linha)
informacoes_divididas = df_meds.loc[df_meds['Diferença entre o órgão + favorável e o - favorável'] >= 80]['Órgão de ATS - Favorável'].apply(lambda x: list(set(x.split('/'))))

# Cria uma lista plana com todas as informações únicas de todas as linhas
todas_informacoes = [info for sublist in informacoes_divididas for info in sublist]

# Conta a frequência de cada informação única
contagem_informacoes = pd.Series(todas_informacoes).value_counts()

contagem_informacoes

NatJus MT             27
TelessaúdeRS-UFRGS    23
NatJus ES             13
NatJus MS             10
NatJus SP              5
NatJus RN              5
NatJus PR              2
TJRS                   2
DMJ                    2
NatJus BA              1
NatJus GO              1
NatJus PA              1
NatJus PE              1
NatJus MA              1
Name: count, dtype: int64

In [14]:
df_meds.loc[df_meds['Diferença entre o órgão + favorável e o - favorável'] == 100]['Órgão de ATS + Favorável']

0      NatJus BA/NatJus MS/NatJus DFT
5                           NatJus MA
6                           NatJus MS
23                          NatJus BA
25                          NatJus BA
49                          NatJus BA
50                          NatJus MA
65                          NatJus BA
74                          NatJus MS
104                         NatJus RJ
107                         NatJus BA
109                         NatJus BA
118                         NatJus BA
122                         NatJus MS
133                         NatJus BA
138                         NatJus BA
143                         NatJus SE
150                         NatJus MS
159                         NatJus BA
161               NatJus BA/NatJus SE
164                         NatJus SE
179                         NatJus BA
181                         NatJus BA
182               NatJus PR/NatJus MS
187                         NatJus SE
227    NatJus DFT/NatJus SE/NatJus PE
240         

In [7]:
# Filtra os dados para o HIAE
hiae_data = df_resultados2[df_resultados2['Órgão de ATS'] == 'NatJus Nacional']

# Lista para armazenar os resultados finais
comparacao_resultados = []

# Itera sobre cada medicamento e CID avaliados pelo HIAE
for index, row in hiae_data.iterrows():
    med = row['Tecnologia']
    cid = row['CID']
    hiae_num = row['Nº de pareceres']
    hiae_taxa = row.get('Favorável', 0)  # Supondo que 'Favorável' é a coluna que indica a taxa de conclusões favoráveis

    # Filtra os dados para outras instituições que avaliaram o mesmo medicamento e CID
    outras_inst_data = df_resultados2[(df_resultados2['Tecnologia'] == med) & (df_resultados2['CID'] == cid) & (df_resultados2['Órgão de ATS'] != 'NatJus Nacional')]

    # Itera sobre as outras instituições
    for _, outra_inst_row in outras_inst_data.iterrows():
        inst = outra_inst_row['Órgão de ATS']
        inst_num = outra_inst_row['Nº de pareceres']
        inst_taxa = outra_inst_row.get('Favorável', 0)

        # Determina quem teve a maior taxa de conclusões favoráveis
        melhor = 'NatJus Nacional' if hiae_taxa > inst_taxa else inst

        # Adiciona os resultados à lista
        comparacao_resultados.append({
            'med': med,
            'cid': cid,
            'inst': inst,
            'inst_num': inst_num,
            'inst_taxa': inst_taxa,
            'NatJusNacional_num': hiae_num,
            'NatJusNacional_taxa': hiae_taxa,
            'melhor': melhor,
            'ganho': abs(hiae_taxa - inst_taxa)
        })

# Converte a lista de resultados em um DataFrame
df_comparacao = pd.DataFrame(comparacao_resultados)

# Salva o DataFrame em um arquivo Excel
df_comparacao.to_excel('comparacao_inst_hiae.xlsx', index=False)

In [25]:
df_comparacao.melhor.value_counts()

melhor
NatJus Nacional       442
DMJ/TJRS              220
NatJus BA             120
NatJus MS              91
NatJus PR              63
NatJus SP              59
NatJus MA              45
NatJus DFT             35
NatJus RJ              35
NatJus SE              34
NatJus GO              24
NatJus RN              21
TelessaúdeRS-UFRGS     17
NatJus PE              16
NatJus MT              16
NatJus ES               9
NatJus PB               2
NatJus AL               2
NatJus AM               1
Name: count, dtype: int64

In [24]:
# Calcular a quantidade de linhas onde a diferença foi de 100
dif_100 = df_comparacao[df_comparacao['ganho'] == 100].shape[0]

# Calcular a quantidade de linhas onde a diferença foi igual ou superior a 90%
dif_80 = df_comparacao[df_comparacao['ganho'] >= 80].shape[0]

# Calcular a quantidade de linhas onde a diferença foi igual ou superior a 50%
dif_50 = df_comparacao[df_comparacao['ganho'] >= 50].shape[0]

# Calcular a quantidade de linhas onde a diferença foi igual ou superior a 50%
dif_20 = df_comparacao[df_comparacao['ganho'] >= 20].shape[0]

# Imprimir os resultados
print(f"Diferença de 100: {dif_100} linhas")
print(f"Diferença igual ou superior a 80%: {dif_80} linhas")
print(f"Diferença igual ou superior a 50%: {dif_50} linhas")
print(f"Diferença igual ou superior a 20%: {dif_20} linhas")
print(len(df_comparacao))

Diferença de 100: 9 linhas
Diferença igual ou superior a 80%: 44 linhas
Diferença igual ou superior a 50%: 220 linhas
Diferença igual ou superior a 20%: 562 linhas
1252


In [8]:
# Lista para armazenar os maiores ganhos por instituição
maiores_ganhos = []

# Itera sobre cada instituição
for inst in df_comparacao['inst'].unique():
    # Filtra os dados para a instituição atual
    inst_data = df_comparacao[df_comparacao['inst'] == inst]
    
    # Encontra o ganho máximo optando pelo NatJus Nacional
    natjus_data = inst_data[inst_data['melhor'] == 'NatJus Nacional']
    if not natjus_data.empty:
        max_ganho_natjus = natjus_data['ganho'].max()
        max_ganho_natjus_data = natjus_data[natjus_data['ganho'] == max_ganho_natjus]
        
        # Tratar empates para o NatJus Nacional
        if len(max_ganho_natjus_data) > 1:
            med_natjus = "/".join(max_ganho_natjus_data['med'].astype(str).unique())
            cid_natjus = "/".join(max_ganho_natjus_data['cid'].astype(str).unique())
            natjus_taxa = "/".join(f"{taxa:.1f}% (n={num})" for taxa, num in zip(max_ganho_natjus_data['NatJusNacional_taxa'], max_ganho_natjus_data['NatJusNacional_num']))
            inst_taxa_values = "/".join(f"{taxa:.1f}% (n={num})" for taxa, num in zip(max_ganho_natjus_data['inst_taxa'], max_ganho_natjus_data['inst_num']))
        else:
            med_natjus = max_ganho_natjus_data['med'].iloc[0]
            cid_natjus = max_ganho_natjus_data['cid'].iloc[0]
            natjus_taxa = f"{max_ganho_natjus_data['NatJusNacional_taxa'].iloc[0]:.1f}% (n={max_ganho_natjus_data['NatJusNacional_num'].iloc[0]})"
            inst_taxa_values = f"{max_ganho_natjus_data['inst_taxa'].iloc[0]:.1f}% (n={max_ganho_natjus_data['inst_num'].iloc[0]})"
        
        maiores_ganhos.append({
            'inst': inst,
            'ganho_escolhendo': 'NatJus Nacional',
            'med': med_natjus,
            'cid': cid_natjus,
            'NatJusNacional_taxa': natjus_taxa,
            'inst_taxa': inst_taxa_values,
            'ganho': max_ganho_natjus
        })
    
    # Encontra o ganho máximo optando pela própria instituição
    inst_melhor_data = inst_data[inst_data['melhor'] == inst]
    if not inst_melhor_data.empty:
        max_ganho_inst = inst_melhor_data['ganho'].max()
        max_ganho_inst_data = inst_melhor_data[inst_melhor_data['ganho'] == max_ganho_inst]
        
        # Tratar empates para a própria instituição
        if len(max_ganho_inst_data) > 1:
            med_inst = "/".join(max_ganho_inst_data['med'].astype(str).unique())
            cid_inst = "/".join(max_ganho_inst_data['cid'].astype(str).unique())
            natjus_taxa_inst = "/".join(f"{taxa:.1f}% (n={num})" for taxa, num in zip(max_ganho_inst_data['NatJusNacional_taxa'], max_ganho_inst_data['NatJusNacional_num']))
            inst_taxa_values_inst = "/".join(f"{taxa:.1f}% (n={num})" for taxa, num in zip(max_ganho_inst_data['inst_taxa'], max_ganho_inst_data['inst_num']))
        else:
            med_inst = max_ganho_inst_data['med'].iloc[0]
            cid_inst = max_ganho_inst_data['cid'].iloc[0]
            natjus_taxa_inst = f"{max_ganho_inst_data['NatJusNacional_taxa'].iloc[0]:.1f}% (n={max_ganho_inst_data['NatJusNacional_num'].iloc[0]})"
            inst_taxa_values_inst = f"{max_ganho_inst_data['inst_taxa'].iloc[0]:.1f}% (n={max_ganho_inst_data['inst_num'].iloc[0]})"
        
        maiores_ganhos.append({
            'inst': inst,
            'ganho_escolhendo': inst,
            'med': med_inst,
            'cid': cid_inst,
            'NatJusNacional_taxa': natjus_taxa_inst,
            'inst_taxa': inst_taxa_values_inst,
            'ganho': max_ganho_inst
        })

# Converte a lista de maiores ganhos em um DataFrame
df_maiores_ganhos = pd.DataFrame(maiores_ganhos)

# Reordena as colunas para colocar 'ganho' à direita
cols = ['inst', 'ganho_escolhendo', 'med', 'cid', 'NatJusNacional_taxa', 'inst_taxa', 'ganho']
df_maiores_ganhos = df_maiores_ganhos[cols]

# Salva o DataFrame em um arquivo Excel
df_maiores_ganhos.to_excel('maiores_ganhos_inst_e_natjus.xlsx', index=False)

In [9]:
# Lista para armazenar os maiores ganhos por instituição
maiores_ganhos = []

# Itera sobre cada instituição
for inst in df_comparacao['inst'].unique():
    # Filtra os dados para a instituição atual
    inst_data = df_comparacao[df_comparacao['inst'] == inst]
    
    # Encontra o ganho máximo optando pelo NatJus Nacional
    natjus_data = inst_data[inst_data['melhor'] == 'NatJus Nacional']
    if not natjus_data.empty:
        max_ganho_natjus = natjus_data['ganho'].max()
        max_ganho_natjus_data = natjus_data[natjus_data['ganho'] == max_ganho_natjus]
        
        # Adiciona cada caso empatado separadamente
        for idx, row in max_ganho_natjus_data.iterrows():
            med_natjus = row['med']
            cid_natjus = row['cid']
            natjus_taxa = f"{row['NatJusNacional_taxa']:.1f}% (n={row['NatJusNacional_num']})"
            inst_taxa_values = f"{row['inst_taxa']:.1f}% (n={row['inst_num']})"
            
            maiores_ganhos.append({
                'inst': inst,
                'ganho_escolhendo': 'NatJus Nacional',
                'med': med_natjus,
                'cid': cid_natjus,
                'NatJusNacional_taxa': natjus_taxa,
                'inst_taxa': inst_taxa_values,
                'ganho': max_ganho_natjus
            })
    
    # Encontra o ganho máximo optando pela própria instituição
    inst_melhor_data = inst_data[inst_data['melhor'] == inst]
    if not inst_melhor_data.empty:
        max_ganho_inst = inst_melhor_data['ganho'].max()
        max_ganho_inst_data = inst_melhor_data[inst_melhor_data['ganho'] == max_ganho_inst]
        
        # Adiciona cada caso empatado separadamente
        for idx, row in max_ganho_inst_data.iterrows():
            med_inst = row['med']
            cid_inst = row['cid']
            natjus_taxa_inst = f"{row['NatJusNacional_taxa']:.1f}% (n={row['NatJusNacional_num']})"
            inst_taxa_values_inst = f"{row['inst_taxa']:.1f}% (n={row['inst_num']})"
            
            maiores_ganhos.append({
                'inst': inst,
                'ganho_escolhendo': inst,
                'med': med_inst,
                'cid': cid_inst,
                'NatJusNacional_taxa': natjus_taxa_inst,
                'inst_taxa': inst_taxa_values_inst,
                'ganho': max_ganho_inst
            })

# Converte a lista de maiores ganhos em um DataFrame
df_maiores_ganhos = pd.DataFrame(maiores_ganhos)

# Reordena as colunas para colocar 'ganho' à direita
cols = ['inst', 'ganho_escolhendo', 'med', 'cid', 'NatJusNacional_taxa', 'inst_taxa', 'ganho']
df_maiores_ganhos = df_maiores_ganhos[cols]

# Salva o DataFrame em um arquivo Excel
df_maiores_ganhos.to_excel('maiores_ganhos_inst_e_natjus.xlsx', index=False)